In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Mount Drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Prepare the event data for predictive modeling 
- Special Treatment to ***Events*** columns: ***agent*** (browser)



In [ ]:
event_user_page = pd.read_csv('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/event_user_pages.csv')

In [ ]:
event_user_page.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'id_x', 'distinct_id', 'current_url',
       'name', 'initial_referrerTrue', 'referrer', 'user_agent',
       'performed_at', 'properties', 'inserted_at_x', 'membership_purchased',
       'total_sessions', 'content_count', 'paywalls_count',
       'recommended_content_clicks', 'searches_performed', 'prior_user',
       'total_events', 'ecommerce_event_count',
       'account_management_event_count', 'total_cta_count', 'total_ugc_count',
       'homepage_viewed_count', 'product_viewed_count', 'referrer_count',
       'restricted_content_count', 'recipe_view_count', 'mr_wizard_view_count',
       'article_view_count', 'resource_view_count', 'cta_impression_count',
       'cta_conversion_count', 'negative_content_viewed_count',
       'mixed_content_viewed_count', 'positive_content_viewed_count',
       'neutral_content_viewed_count', 'id_y', 'url', 'published_at', 'title',
       'dom_count', 'image_count', 'link_count', 'scripts_count',
       'inser

In [ ]:
event_user_page.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'id_x','properties','id_y','current_url','url','title','inserted_at_y'],inplace=True)

In [ ]:
event_user_page = event_user_page.rename(columns={'initial_referrerTrue':'initial_referrer','inserted_at_x':'inserted_at'})

In [ ]:
event_user_page.head()

,distinct_id,name,initial_referrer,referrer,user_agent,performed_at,inserted_at,membership_purchased,total_sessions,content_count,paywalls_count,recommended_content_clicks,searches_performed,prior_user,total_events,ecommerce_event_count,account_management_event_count,total_cta_count,total_ugc_count,homepage_viewed_count,product_viewed_count,referrer_count,restricted_content_count,recipe_view_count,mr_wizard_view_count,article_view_count,resource_view_count,cta_impression_count,cta_conversion_count,negative_content_viewed_count,mixed_content_viewed_count,positive_content_viewed_count,neutral_content_viewed_count,published_at,dom_count,image_count,link_count,scripts_count,updated_at
0,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:00 AM,2019/10/24 6:16:21 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,521.0,9.0,83.0,50.0,9/26/20 19:10
1,adcc583b-c354-4e5d-ba77-21aa57362a57,One Page Checkout,https://www.google.com/,https://byo.com/,Mozilla/5.0 (Linux; Android 9; SM-G960F) Apple...,4:39:01 AM,2019/10/24 6:17:57 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0,NaN,540.0,16.0,26.0,61.0,12/10/20 18:37
2,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:01 AM,2019/10/24 6:19:47 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,609.0,10.0,85.0,49.0,12/11/20 3:32
3,adcc583b-c354-4e5d-ba77-21aa57362a57,Search Performed,https://www.google.com/,https://byo.com/,Mozilla/5.0 (Linux; Android 9; SM-G960F) Apple...,4:39:01 AM,2019/10/24 6:20:20 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0,NaN,640.0,9.0,150.0,43.0,12/11/20 6:31
4,adcc583b-c354-4e5d-ba77-21aa57362a57,Search Performed,https://www.google.com/,https://byo.com/articles/,Mozilla/5.0 (Linux; Android 9; SM-G960F) Apple...,4:39:01 AM,2019/10/24 6:20:26 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0,NaN,770.0,11.0,85.0,43.0,10/24/19 18:20


# Recode - Extract the useful column content

*   Delete "AM" from 'performed at' - all of them are AMs


In [ ]:
event_user_page['performed_at'].head()

0    4:39:00 AM
1    4:39:01 AM
2    4:39:01 AM
3    4:39:01 AM
4    4:39:01 AM
Name: performed_at, dtype: object

In [ ]:
am_count=event_user_page['performed_at'].str.contains('AM').sum()
if am_count>0:
    print ("There are {m} AMs".format(m=am_count))

There are 687102 AMs


In [ ]:
event_user_page['performed_at'] = event_user_page['performed_at'].str.replace(" AM", "")

In [ ]:
event_user_page['performed_at'].value_counts()

6:17:16        4791
7:14:52        4195
7:58:04        4090
8:15:21        3659
5:32:38        3256
               ... 
11:24:19          1
12:07:34 PM       1
6:11:42           1
4:42:36           1
6:59:01           1
Name: performed_at, Length: 37303, dtype: int64

*   Add new column **'channel'** to recode **'initial_referrer'** - i.e. all websites containing "google" to 'google'; 

  **But should keep the original columns**, why? address name like google.uk, it may tells us **geolocation** information.

In [ ]:
event_user_page['channel'] = event_user_page['initial_referrer'].copy()

In [ ]:
event_user_page.head()

,distinct_id,name,initial_referrer,referrer,user_agent,performed_at,inserted_at,membership_purchased,total_sessions,content_count,paywalls_count,recommended_content_clicks,searches_performed,prior_user,total_events,ecommerce_event_count,account_management_event_count,total_cta_count,total_ugc_count,homepage_viewed_count,product_viewed_count,referrer_count,restricted_content_count,recipe_view_count,mr_wizard_view_count,article_view_count,resource_view_count,cta_impression_count,cta_conversion_count,negative_content_viewed_count,mixed_content_viewed_count,positive_content_viewed_count,neutral_content_viewed_count,published_at,dom_count,image_count,link_count,scripts_count,updated_at,channel
0,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:00,2019/10/24 6:16:21 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,521.0,9.0,83.0,50.0,9/26/20 19:10,https://www.google.com/
1,adcc583b-c354-4e5d-ba77-21aa57362a57,One Page Checkout,https://www.google.com/,https://byo.com/,Mozilla/5.0 (Linux; Android 9; SM-G960F) Apple...,4:39:01,2019/10/24 6:17:57 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0,NaN,540.0,16.0,26.0,61.0,12/10/20 18:37,https://www.google.com/
2,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:01,2019/10/24 6:19:47 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,609.0,10.0,85.0,49.0,12/11/20 3:32,https://www.google.com/
3,adcc583b-c354-4e5d-ba77-21aa57362a57,Search Performed,https://www.google.com/,https://byo.com/,Mozilla/5.0 (Linux; Android 9; SM-G960F) Apple...,4:39:01,2019/10/24 6:20:20 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0,NaN,640.0,9.0,150.0,43.0,12/11/20 6:31,https://www.google.com/
4,adcc583b-c354-4e5d-ba77-21aa57362a57,Search Performed,https://www.google.com/,https://byo.com/articles/,Mozilla/5.0 (Linux; Android 9; SM-G960F) Apple...,4:39:01,2019/10/24 6:20:26 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0,NaN,770.0,11.0,85.0,43.0,10/24/19 18:20,https://www.google.com/


In [ ]:
event_user_page[['channel','initial_referrer']]

,channel,initial_referrer
0,https://www.google.com/,https://www.google.com/
1,https://www.google.com/,https://www.google.com/
2,https://www.google.com/,https://www.google.com/
3,https://www.google.com/,https://www.google.com/
4,https://www.google.com/,https://www.google.com/
...,...,...
1239504,https://www.google.com/,https://www.google.com/
1239505,https://www.google.com,https://www.google.com
1239506,https://www.google.com,https://www.google.com
1239507,https://www.google.com,https://www.google.com


In [ ]:
event_user_page['channel'].value_counts().to_frame().head(30)

,channel
https://www.google.com/,410573
https://www.google.com,53891
https://www.bing.com/,10269
https://duckduckgo.com/,7769
https://www.google.co.uk,7410
https://www.google.com.au,5477
https://www.google.ca/,5429
https://www.google.ca,5271
https://www.google.co.uk/,5072
https://l.facebook.com/,4936


In [ ]:
x = event_user_page['channel'].tolist()

In [ ]:
tags = ['google', 'reddit', 'pinterest','yahoo','bing','facebook','duckduckgo','byo.com','youtube','baidu','instagram','wikipedia','ecosia.org',
        'brewersfriend','qwant','homebrewtalk','startpage','yandex','surveymonkey','symbaloo','hominilupulo','homebrewersassociation'
        'beeradvocate','lavasoft.gosearchresults.com','beeradvocate','ratebeer','nortonsafe','norbrygg','t.co','diybeer',
        'cerveceros-caseros','kaarlonkotipanimo','beer-deli','scottjanish','beersmith']

for tag in tags:
  event_user_page.loc[event_user_page['channel'].str.contains(tag, case=False,na=False), 'channel'] = tag

In [ ]:
tags = pd.DataFrame(event_user_page['channel'].value_counts())

In [ ]:
tags.shape

(1056, 1)

In [ ]:
event_user_page.channel.value_counts().to_frame().head(50)

,channel
google,524173
byo.com,17084
facebook,14289
bing,11343
duckduckgo,9878
yahoo,4760
pinterest,1821
t.co,1704
homebrewtalk,1021
instagram,708


In [ ]:
s = event_user_page.channel.value_counts().to_frame().head(50).sum()/event_user_page.channel.value_counts().to_frame().sum()*100
print("The top 50 channels covered almost %.2f%% total events."%s)

The top 50 channels covered almost 99.21% total events.


# Add 'source' dimension - displaying the categories of the sources
- We can categorize the 50 channels into sources: social media, search engine, brew/cuisine website, BYO's website

In [ ]:
social_media = ['reddit', 'pinterest''facebook','youtube','instagram','wikipedia','t.co']
search_engine = ['google','yahoo','bing','duckduckgo','baidu','ecosia','qwant','startpage','yandex','surveymonkey','nortonsafe','symbaloo','lavasoft.gosearchresults.com','https://t.umblr.com/','http://dnserrorassist.att.net/search/?q=http%3A//byocamp.com/&r=&bc=']
internal_link = ['byo.com']
brew_website = ['brewersfriend','homebrewtalk','https://www.beeradvocate.com/','homebrewersassociation','hominilupulo','beeradvocate','ratebeer','diybeer','beer-deli','scottjanish','beersmith','kaarlonkotipanimo','cerveceros-caseros',
                'norbrygg','beer-deli','https://www.hobbybrouwen.nl/','ratebeer','https://www.piwo.org/','https://hobbybrauer.de/','https://brygklubben.dk/','https://www.cervezartesana.es/blog/post/el-estilo-kolsch-segun-la-bjcp.html',
                'https://www.brassageamateur.com/','https://www.crankandstein.net/','https://www.buvosszakacs.com/2015/02/05/onkepzokor-diacetil/	']

In [ ]:
len(social_media)+len(search_engine)+len(internal_link)+len(brew_website)
# The 4 left either are broken links or have no value. 

46

In [ ]:
event_user_page["source"] = None
event_user_page.loc[event_user_page["channel"].isin(social_media), "source"] = "Social Media"
event_user_page.loc[event_user_page["channel"].isin(search_engine), "source"] = "Search Engine"
event_user_page.loc[event_user_page["channel"].isin(internal_link), "source"] = "Internal Link"
event_user_page.loc[event_user_page["channel"].isin(brew_website), "source"] = "Brew Website"

In [ ]:
event_user_page.loc[~event_user_page["channel"].isin(brew_website+social_media + search_engine + internal_link),"source"] = "Others"

In [ ]:
event_user_page.reset_index(inplace=True)

In [ ]:
# Display number of events performed by users via different sources 
tryone = pd.DataFrame(event_user_page.groupby(['distinct_id','source'])['index'].agg(['count']).sort_values('count',ascending=False))

In [ ]:
tryone.reset_index(inplace=True)

In [ ]:
tryone.shape

(293850, 3)

In [ ]:
tryone['distinct_id'].unique().shape[0]

293845

In [ ]:
tryone.reset_index(inplace=True)

In [ ]:
# How many users visit the websites via multiple (>1) sources?
trytwo = tryone.groupby(['distinct_id'])['index'].agg('count').reset_index()

In [ ]:
# who are they?
multiplesource = trytwo[trytwo['index'] != 1]['distinct_id'].unique()

In [ ]:
trythree = event_user_page[event_user_page['distinct_id'].isin(multiplesource)][['index','distinct_id','source']]

In [ ]:
# 2 people
trythree.groupby(['distinct_id','source'])['index'].agg('count').reset_index()

,distinct_id,source,index
0,3f2fa233-444b-4e87-a5c4-0277499c4be4,Internal Link,4
1,3f2fa233-444b-4e87-a5c4-0277499c4be4,Others,358431
2,3f2fa233-444b-4e87-a5c4-0277499c4be4,Search Engine,77
3,3f2fa233-444b-4e87-a5c4-0277499c4be4,Social Media,26
4,a5c40277-499c-4be4-bd73-ee3ba36f6585,Others,45212
5,a5c40277-499c-4be4-bd73-ee3ba36f6585,Search Engine,8
6,a5c40277-499c-4be4-bd73-ee3ba36f6585,Social Media,1


# User Age

In [ ]:
event_user_page['inserted_date'] = pd.to_datetime(event_user_page['inserted_at']).dt.strftime('%Y-%m-%d')
earliest_date = pd.DataFrame(event_user_page.groupby(['distinct_id'])['inserted_date'].agg(['min']))
event_user_page = event_user_page.merge(earliest_date,how='left',on='distinct_id')
import datetime as dt
today=dt.datetime(2021,3,12)
event_user_page['min']=pd.to_datetime(event_user_page['min'],yearfirst=True)

In [ ]:
today

datetime.datetime(2021, 3, 12, 0, 0)

In [ ]:
event_user_page['User Age'] = pd.DataFrame(today - event_user_page['min'])

In [ ]:
event_user_page['User Age']

0         505 days
1         505 days
2         505 days
3         505 days
4         505 days
            ...   
1239504    66 days
1239505    66 days
1239506    66 days
1239507    66 days
1239508   298 days
Name: User Age, Length: 1239509, dtype: timedelta64[ns]

# Performed At

In [ ]:
event_user_page.performed_at.value_counts()

6:17:16        4791
7:14:52        4195
7:58:04        4090
8:15:21        3659
5:32:38        3256
               ... 
11:24:19          1
12:07:34 PM       1
6:11:42           1
4:42:36           1
6:59:01           1
Name: performed_at, Length: 37303, dtype: int64

In [ ]:
event_user_page['Hour'] = pd.to_datetime(event_user_page['performed_at']).apply(lambda x: x.hour)

In [ ]:
event_user_page['Hour'].value_counts()

14    247613
13    197491
8     132987
7     127587
5     120868
6     116388
9     105331
12     94909
11     35594
10     24260
4      24070
15     12385
1         14
16         9
3          3
Name: Hour, dtype: int64

In [ ]:
def sceen_time(series):
    if 4 <= series < 12:
        return "Morning"
    elif 12 <= series < 18:
        return "Afternoon"
    elif 18 <= series < 22:
        return "Evening"
    elif 22 <= series < 24:
        return "Night"
    elif 0 <= series <4:
        return "Night"

event_user_page['ScreenTime'] = event_user_page['Hour'].apply(sceen_time)

In [ ]:
# extract the majority screentime of each user
event_try = event_user_page.groupby(['distinct_id'])['ScreenTime'].value_counts().reset_index(name='screentime_count')
ScreenTime = event_try.groupby(['distinct_id'])['ScreenTime'].max().reset_index()

In [ ]:
ScreenTime

,distinct_id,ScreenTime
0,000028db-5119-4edd-a73c-b2441e626c65,Afternoon
1,0000336c-fa7f-463f-8080-8d59288c3ae5,Morning
2,000037df-a0f8-4fab-971b-155b548cac9e,Afternoon
3,0000d24c-e8af-4de2-8792-ad2269315785,Afternoon
4,00011087-656b-4e66-bfd5-1a6725b27c5c,Afternoon
...,...,...
293840,ffff165d-fcbb-4227-9c39-2ccab69d9a63,Afternoon
293841,ffff33f7-0e6c-4119-a3fc-7c54dd32196a,Afternoon
293842,ffff53ac-5d34-436c-8baa-8f931dab1dcc,Afternoon
293843,ffff6d60-7ba1-4b22-8b23-70130b506ec5,Morning


In [ ]:
# Display number of events performed by users via different sources 
#event_user_page.reset_index(inplace=True)
#tryone = pd.DataFrame(event_user_page.groupby(['distinct_id','ScreenTime'])['index'].agg(['count']).sort_values('count',ascending=False))
#tryone.reset_index(inplace=True)
#tryone.shape
#tryone['distinct_id'].unique().shape[0]
#tryone.reset_index(inplace=True)

# How many users visit the websites via multiple (>1) sources?
#trytwo = tryone.groupby(['distinct_id'])['index'].agg('count').reset_index()
# who are they?
#multiplesource = trytwo[trytwo['index'] != 1]['distinct_id'].unique()
#trythree = event_user_page[event_user_page['distinct_id'].isin(multiplesource)][['index','distinct_id','ScreenTime']]
# 2 people
#tryfour = trythree.groupby(['distinct_id','ScreenTime'])['index'].agg('count').reset_index()
# tryfour.sort_values('index', ascending=False).drop_duplicates(['distinct_id','ScreenTime'],inplace=True)

In [ ]:
#ScreenTime = tryfour.groupby(['distinct_id'])['index'].transform('max') == tryfour['index']

In [ ]:
#ready_to_join = tryfour[ScreenTime]

In [ ]:
#tryfour[tryfour['distinct_id']=='00069dd9-daf1-422d-8719-7290a9d733be']

In [ ]:
# good -> ready_to_join
#distinct_id_list = good["distinct_id"].tolist()
#for distinct_id in distinct_id_list:
#  event_user_page.loc[event_user_page['distinct_id'].str.contains(distinct_id, case=False,na=False), 'ScreenTime'] = good[good['distinct_id']==distinct_id]['ScreenTime']

# Agent (browser & device)

In [ ]:
event_user_page.columns


Index(['index', 'distinct_id', 'name', 'initial_referrer', 'referrer',
       'user_agent', 'performed_at', 'inserted_at', 'membership_purchased',
       'total_sessions', 'content_count', 'paywalls_count',
       'recommended_content_clicks', 'searches_performed', 'prior_user',
       'total_events', 'ecommerce_event_count',
       'account_management_event_count', 'total_cta_count', 'total_ugc_count',
       'homepage_viewed_count', 'product_viewed_count', 'referrer_count',
       'restricted_content_count', 'recipe_view_count', 'mr_wizard_view_count',
       'article_view_count', 'resource_view_count', 'cta_impression_count',
       'cta_conversion_count', 'negative_content_viewed_count',
       'mixed_content_viewed_count', 'positive_content_viewed_count',
       'neutral_content_viewed_count', 'published_at', 'dom_count',
       'image_count', 'link_count', 'scripts_count', 'updated_at', 'channel',
       'source', 'inserted_date', 'min_x', 'min_y', 'User Age', 'Hour',
       'Scr

In [ ]:
pd.options.display.max_colwidth = 300
event_user_page.user_agent.head(20)

0                   Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.116 Safari/537.36
1                Mozilla/5.0 (Linux; Android 9; SM-G960F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.116 Mobile Safari/537.36
2                   Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.116 Safari/537.36
3                Mozilla/5.0 (Linux; Android 9; SM-G960F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.116 Mobile Safari/537.36
4                Mozilla/5.0 (Linux; Android 9; SM-G960F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.116 Mobile Safari/537.36
5                      Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36
6                       Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36
7               Mozilla/5.0

In [ ]:
event_user_page['browser'] = event_user_page['user_agent'].copy()

In [ ]:
event_user_page['device'] = event_user_page['user_agent'].copy()

In [ ]:
devices = ['Linux','iPhone','Macintosh','Windows','iPad','compatible','AdsBot-Google']

In [ ]:
browsers = ['Chrome','Firefox','Safari','Edg']

In [ ]:
for browser in browsers:
    event_user_page.loc[event_user_page['browser'].str.contains(browser, case=False,na=False), 'browser'] = browser

In [ ]:
others = ['Android','OmniWeb','e.ventures','Wappalyzer',"/"]
for item in others:
    event_user_page.loc[event_user_page['browser'].str.contains(item, case=False,na=False), 'browser'] = 'Others'

In [ ]:
event_user_page.browser.value_counts()

Chrome     888566
Safari     270290
Firefox     74298
Others       6355
Name: browser, dtype: int64

In [ ]:
for device in devices:
    event_user_page.loc[event_user_page['device'].str.contains(device, case=False,na=False), 'device'] = device

In [ ]:
event_user_page.device.value_counts(dropna=False)

Linux                                                                                                                   449094
Windows                                                                                                                 327526
compatible                                                                                                              167267
iPhone                                                                                                                  146227
Macintosh                                                                                                               119752
                                                                                                                         ...  
Mozilla/5.0 (X11; CrOS x86_64 10575.58.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36           1
Mozilla/5.0 (X11; CrOS x86_64 12739.105.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.158 Safari/5

In [ ]:
others = ['Android','OmniWeb','e.ventures','Wappalyzer',"/",'AdsBot-Google','Firefox']
for item in others:
    event_user_page.loc[event_user_page['device'].str.contains(item,case=False,na=False),'device'] = 'Others'

In [ ]:
event_user_page.device.value_counts(dropna=False)

Linux         449094
Windows       327526
compatible    167267
iPhone        146227
Macintosh     119752
iPad           15869
Others         13774
Name: device, dtype: int64

In [ ]:
event_user_page.head()

,index,distinct_id,name,initial_referrer,referrer,user_agent,performed_at,inserted_at,membership_purchased,total_sessions,content_count,paywalls_count,recommended_content_clicks,searches_performed,prior_user,total_events,ecommerce_event_count,account_management_event_count,total_cta_count,total_ugc_count,homepage_viewed_count,product_viewed_count,referrer_count,restricted_content_count,recipe_view_count,mr_wizard_view_count,article_view_count,resource_view_count,cta_impression_count,cta_conversion_count,negative_content_viewed_count,mixed_content_viewed_count,positive_content_viewed_count,neutral_content_viewed_count,published_at,dom_count,image_count,link_count,scripts_count,updated_at,channel,source,inserted_date,min_x,min_y,User Age,Hour,ScreenTime,browser,device
0,0,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,"Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.116 Safari/537.36",4:39:00,2019/10/24 6:16:21 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,521.0,9.0,83.0,50.0,9/26/20 19:10,google,Search Engine,2019-10-24,2019-10-24,2019-10-24,505 days,4,Morning,Chrome,Linux
1,1,adcc583b-c354-4e5d-ba77-21aa57362a57,One Page Checkout,https://www.google.com/,https://byo.com/,"Mozilla/5.0 (Linux; Android 9; SM-G960F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.116 Mobile Safari/537.36",4:39:01,2019/10/24 6:17:57 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0,NaN,540.0,16.0,26.0,61.0,12/10/20 18:37,google,Search Engine,2019-10-24,2019-10-24,2019-10-24,505 days,4,Morning,Chrome,Linux
2,2,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,"Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.116 Safari/537.36",4:39:01,2019/10/24 6:19:47 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,609.0,10.0,85.0,49.0,12/11/20 3:32,google,Search Engine,2019-10-24,2019-10-24,2019-10-24,505 days,4,Morning,Chrome,Linux
3,3,adcc583b-c354-4e5d-ba77-21aa57362a57,Search Performed,https://www.google.com/,https://byo.com/,"Mozilla/5.0 (Linux; Android 9; SM-G960F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.116 Mobile Safari/537.36",4:39:01,2019/10/24 6:20:20 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0,NaN,640.0,9.0,150.0,43.0,12/11/20 6:31,google,Search Engine,2019-10-24,2019-10-24,2019-10-24,505 days,4,Morning,Chrome,Linux
4,4,adcc583b-c354-4e5d-ba77-21aa57362a57,Search Performed,https://www.google.com/,https://byo.com/articles/,"Mozilla/5.0 (Linux; Android 9; SM-G960F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.116 Mobile Safari/537.36",4:39:01,2019/10/24 6:20:26 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0,NaN,770.0,11.0,85.0,43.0,10/24/19 18:20,google,Search Engine,2019-10-24,2019-10-24,2019-10-24,505 days,4,Morning,Chrome,Linux


In [ ]:
# extract the majority channel of each user
event_try = event_user_page.groupby(['distinct_id'])['channel'].value_counts().reset_index(name='channel_count')
channel = event_try.groupby(['distinct_id'])['channel'].max().reset_index()
channel.head()

,distinct_id,channel
0,000028db-5119-4edd-a73c-b2441e626c65,google
1,000037df-a0f8-4fab-971b-155b548cac9e,google
2,0000d24c-e8af-4de2-8792-ad2269315785,google
3,00011087-656b-4e66-bfd5-1a6725b27c5c,google
4,00019adf-e5f4-4bcf-b2cc-bd3676c8088b,google


In [ ]:
# extract the majority source of each user
event_try = event_user_page.groupby(['distinct_id'])['source'].value_counts().reset_index(name='source_count')
source = event_try.groupby(['distinct_id'])['source'].max().reset_index()
source.head()

,distinct_id,source
0,000028db-5119-4edd-a73c-b2441e626c65,Search Engine
1,0000336c-fa7f-463f-8080-8d59288c3ae5,Others
2,000037df-a0f8-4fab-971b-155b548cac9e,Search Engine
3,0000d24c-e8af-4de2-8792-ad2269315785,Search Engine
4,00011087-656b-4e66-bfd5-1a6725b27c5c,Search Engine


In [ ]:
# extract the majority browser of each user
event_try = event_user_page.groupby(['distinct_id'])['browser'].value_counts().reset_index(name='browser_count')
browser = event_try.groupby(['distinct_id'])['browser'].max().reset_index()
browser.head()

,distinct_id,browser
0,000028db-5119-4edd-a73c-b2441e626c65,Safari
1,0000336c-fa7f-463f-8080-8d59288c3ae5,Chrome
2,000037df-a0f8-4fab-971b-155b548cac9e,Safari
3,0000d24c-e8af-4de2-8792-ad2269315785,Safari
4,00011087-656b-4e66-bfd5-1a6725b27c5c,Safari


In [ ]:
# extract the majority device of each user
event_try = event_user_page.groupby(['distinct_id'])['device'].value_counts().reset_index(name='device_count')
device = event_try.groupby(['distinct_id'])['device'].max().reset_index()
device.head()

,distinct_id,device
0,000028db-5119-4edd-a73c-b2441e626c65,iPhone
1,0000336c-fa7f-463f-8080-8d59288c3ae5,Windows
2,000037df-a0f8-4fab-971b-155b548cac9e,Macintosh
3,0000d24c-e8af-4de2-8792-ad2269315785,iPhone
4,00011087-656b-4e66-bfd5-1a6725b27c5c,iPhone


In [ ]:
# user age
age = event_user_page.groupby(['distinct_id'])['User Age'].value_counts().reset_index(name='user_count')
age =age[['distinct_id','User Age']]

In [ ]:
age['User Age'] = age['User Age'] / np.timedelta64(1, 'D')

In [ ]:
age.head()

,distinct_id,User Age
0,000028db-5119-4edd-a73c-b2441e626c65,156.0
1,0000336c-fa7f-463f-8080-8d59288c3ae5,211.0
2,000037df-a0f8-4fab-971b-155b548cac9e,174.0
3,0000d24c-e8af-4de2-8792-ad2269315785,126.0
4,00011087-656b-4e66-bfd5-1a6725b27c5c,131.0


In [ ]:
db = device.merge(browser, on='distinct_id')
sdb = db.merge(source,on='distinct_id') 
csdb = sdb.merge(channel,on='distinct_id')
scsdb = csdb.merge(ScreenTime,on='distinct_id')
ascsdb = scsdb.merge(age,on='distinct_id')

In [ ]:
ascsdb

,distinct_id,device,browser,source,channel,ScreenTime,User Age
0,000028db-5119-4edd-a73c-b2441e626c65,iPhone,Safari,Search Engine,google,Afternoon,156.0
1,000037df-a0f8-4fab-971b-155b548cac9e,Macintosh,Safari,Search Engine,google,Afternoon,174.0
2,0000d24c-e8af-4de2-8792-ad2269315785,iPhone,Safari,Search Engine,google,Afternoon,126.0
3,00011087-656b-4e66-bfd5-1a6725b27c5c,iPhone,Safari,Search Engine,google,Afternoon,131.0
4,00019adf-e5f4-4bcf-b2cc-bd3676c8088b,Linux,Chrome,Search Engine,google,Afternoon,140.0
...,...,...,...,...,...,...,...
247558,fffeaa26-ce52-469a-ae27-d9679d2a4130,iPhone,Safari,Search Engine,google,Afternoon,69.0
247559,ffff165d-fcbb-4227-9c39-2ccab69d9a63,Linux,Chrome,Search Engine,google,Afternoon,89.0
247560,ffff53ac-5d34-436c-8baa-8f931dab1dcc,iPhone,Safari,Search Engine,google,Afternoon,158.0
247561,ffff6d60-7ba1-4b22-8b23-70130b506ec5,Windows,Chrome,Search Engine,google,Morning,301.0


In [ ]:
ascsdb.to_csv('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/event_data_point.csv')

# User browsing time period

In [ ]:
event_user_page.performed_at

0             4:39:00
1             4:39:01
2             4:39:01
3             4:39:01
4             4:39:01
              ...    
1239504    3:10:17 PM
1239505    3:10:19 PM
1239506    3:10:19 PM
1239507    3:10:18 PM
1239508    3:10:19 PM
Name: performed_at, Length: 1239509, dtype: object

In [ ]:
  # read user data
user_content=pd.read_csv('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/user_content.csv')
# read cluster table
user_cluster6=pd.read_csv('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/user_cluster6.csv')

In [ ]:
user_cluster6

,Unnamed: 0,id,cluster6
0,1,0000336c-fa7f-463f-8080-8d59288c3ae5,0
1,28,00069dd9-daf1-422d-8719-7290a9d733be,5
2,29,0006a15a-268a-4133-b883-58f7e2e70105,5
3,31,0006ce57-eaf5-4b4c-a657-6243ac54728d,0
4,41,0008ee03-4a8b-47b9-a4ae-26aa18a2c925,0
...,...,...,...
24123,281154,fff1880a-fa21-4a3c-ac46-a563c1d744cd,0
24124,281166,fff49477-07c7-441c-b5bd-3ec3a0608a85,0
24125,281176,fff79ba3-8256-471f-9ff1-27a69b1dd8c9,0
24126,281177,fff7a1de-9288-4005-ab61-f3d460e8751f,3


In [ ]:
Y1_train=user_cluster6['cluster6']
train_preds1=user_cluster6['Unnamed: 0']
train_preds1=np.array(train_preds1)


In [ ]:
user_cluster = user_content.merge(user_cluster6,on='id',how='left')
user_cluster = user_cluster[['id','Content','content','cluster6']]

In [ ]:
user_cluster['cluster6']=user_cluster['cluster6'].fillna(6)

In [ ]:
user_cluster['cluster6'].value_counts()

6.0    257073
0.0     15494
5.0      4573
2.0      1570
3.0       957
1.0       952
4.0       582
Name: cluster6, dtype: int64